In [ ]:
import numpy as np
import pandas as pd
from dowhy import CausalModel
import graphviz
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
# EconML imports
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter
import seaborn as sns
import dowhy.plotter
%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')



# The Direct Acyclic Graph

G=graphviz.Digraph()
G.edge("ta_h" , "tr")
G.edge("vel_h" , "ta_h")
G.edge("rh" , "ta_h")
G.edge("ta_h" , "clo")
G.edge("ta_h" , "thermal_sensation")
G.edge("vel_h" , "tr")
G.edge("rh" , "tr")
G.edge("tr" , "clo")
G.edge("tr" , "thermal_sensation")
G.edge("vel_h" , "clo")
G.edge("vel_h" , "thermal_sensation")
G.edge("rh" , "clo")
G.edge("rh" , "thermal_sensation")
G.edge("met" , "thermal_sensation")
G.edge("clo" , "thermal_sensation")

######################################
#########################################

G.edge("blind_curtain" , "vel_h")
G.edge("blind_curtain" , "ta_h")
G.edge("blind_curtain" , "thermal_sensation")




# print(dot.source)

G.format = 'pdf'
G.render(directory='DAG', view = False).replace('\\', '/')



In [3]:
# Reading the data
data = pd.read_excel("data.xlsx")

data_for_causal = data[["ta_h","tr", "vel_h", "rh", "met", "clo", "thermal_sensation", "blind_curtain", "gender"]]
data_for_causal = data_for_causal.dropna()

In [4]:
data_for_causal.groupby('gender').size()

gender
female    161
male       34
dtype: int64

In [5]:
blind_curtain = data_for_causal.copy()
# blind_curtain = blind_curtain.loc[blind_curtain['gender'] == 'male']
blind_curtain = blind_curtain.drop(labels='gender', axis=1)

In [6]:
data_for_causal.shape, blind_curtain.shape

In [7]:
# Initializing causal model
model = CausalModel(data=blind_curtain,
                     graph=G.source.replace("\t", ' ').replace("\n", ' '),
                     treatment="blind_curtain",
                     outcome="thermal_sensation")

# Identifying the estimation method
identified_estimand= model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
       d                             
────────────────(E[thermalₛₑₙₛₐₜᵢₒₙ])
d[blind_curtain]                     
Estimand assumption 1, Unconfoundedness: If U→{blind_curtain} and U→thermal_sensation then P(thermal_sensation|blind_curtain,,U) = P(thermal_sensation|blind_curtain,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [8]:
random_state = 120
dml_estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.econml.dml.DML", # Calling EconMl double machine learning algorithm
                                     control_value = 1,
                                     treatment_value = 0,
                                     target_units = 'ate',
                                     confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              'model_t': GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              "model_final":LassoCV(fit_intercept=False, random_state=random_state),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True),
                                                              'random_state':random_state},
                                               "fit_params":{}})
print(dml_estimate.value)

0.0


In [9]:
# Random cause
res_random=model.refute_estimate(identified_estimand, dml_estimate, method_name="random_common_cause", random_seed=123)
print(res_random)

# Add Unobserved Common Causes
res_unobserved=model.refute_estimate(identified_estimand, dml_estimate, method_name="add_unobserved_common_cause",
                                     confounders_effect_on_treatment="linear", confounders_effect_on_outcome="linear",
                                    effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02, random_seed=123)
print(res_unobserved)

# Placebo Treatment
res_placebo=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="placebo_treatment_refuter", random_seed=123)
print(res_placebo)


# Data Subsets Validation
res_subset=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="data_subset_refuter", subset_fraction=0.8,
        num_simulations=10, random_seed=123)
print(res_subset)

Refute: Add a random common cause
Estimated effect:0.0
New effect:-3.015819419893195e-17
p value:1.0

Refute: Add an Unobserved Common Cause
Estimated effect:0.0
New effect:-6.841324765813922e-17

Refute: Use a Placebo Treatment
Estimated effect:0.0
New effect:0.0
p value:1.0

Refute: Use a subset of data
Estimated effect:0.0
New effect:-0.720089285763185
p value:1.158472873186156e-09

